Download files from google drive using terminal commands

In [1]:
!pip install gdown

In [2]:
!gdown https://drive.google.com/u/0/uc?id=1wWsrN2Ep7x6lWqOXfr4rpKGYrJhWc8z7  # for files
# gdown <file_id>                                 # alternative format
# gdown --folder https://drive.google.com/drive/folders/<file_id>  # for folders
# gdown --folder --id <file_id>                                   # this format works for folders too


Downloading...
From: https://drive.google.com/u/0/uc?id=1wWsrN2Ep7x6lWqOXfr4rpKGYrJhWc8z7
To: /content/speech-emotion-recognition-ravdess-data.zip
100% 78.1M/78.1M [00:01<00:00, 54.8MB/s]


In [3]:
!unzip speech-emotion-recognition-ravdess-data.zip > op # suppressing the terminal output

Import necessary packages

In [4]:
import librosa
import soundfile
import os, glob, pickle
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import accuracy_score

In [16]:
from tqdm import tqdm

Will work with three features-
- **mfcc**: Mel Frequency Cepstral Coefficient, represents the short-term power spectrum of a sound
- **chroma**: Pertains to the 12 different pitch classes
- **mel**: Mel Spectrogram Frequency

In [11]:
def extract_feature(filename, mfcc, chroma, mel):
  with soundfile.SoundFile(filename) as sound_file:
    X = sound_file.read(dtype="float32")
    sample_rate=sound_file.samplerate # sampling rate of the sound file
    if chroma==True:
        stft=np.abs(librosa.stft(X))
    result=np.array([])
    if mfcc==True:
        mfccs=np.mean(librosa.feature.mfcc(y=X, sr=sample_rate, n_mfcc=40).T, axis=0)
        result=np.hstack((result, mfccs))
    if chroma==True:
        chroma=np.mean(librosa.feature.chroma_stft(S=stft, sr=sample_rate).T,axis=0)
        result=np.hstack((result, chroma))
    if mel==True:
        mel=np.mean(librosa.feature.melspectrogram(y=X, sr=sample_rate).T,axis=0)
        result=np.hstack((result, mel))
  return result

In [12]:
emotions={
  '01':'neutral',
  '02':'calm',
  '03':'happy',
  '04':'sad',
  '05':'angry',
  '06':'fearful',
  '07':'disgust',
  '08':'surprised'
}

In [13]:
observed_emotions=['calm', 'happy', 'fearful', 'disgust']

In [17]:
def load_data(test_size=0.2):
    x,y=[],[]
    for file in tqdm(glob.glob("Actor_*/*.wav")):
        file_name=os.path.basename(file)
        emotion=emotions[file_name.split("-")[2]]
        if emotion not in observed_emotions:
            continue
        feature=extract_feature(file, mfcc=True, chroma=True, mel=True)
        x.append(feature)
        y.append(emotion)
    return train_test_split(np.array(x), y, test_size=test_size, random_state=9)

In [18]:
x_train,x_test,y_train,y_test=load_data(test_size=0.25)

100%|██████████| 1440/1440 [00:41<00:00, 34.33it/s]


In [19]:
print((x_train.shape[0], x_test.shape[0]))

(576, 192)


In [37]:
x_train.shape

(576, 180)

In [38]:
x_test.shape

(192, 180)

In [20]:
print(f'Features extracted: {x_train.shape[1]}')

Features extracted: 180


Using an MLP for classification

In [39]:
model=MLPClassifier(alpha=0.01,
                    batch_size=256,
                    epsilon=1e-08,
                    hidden_layer_sizes=(600,300,),
                    learning_rate='adaptive',
                    max_iter=500)

In [40]:
model.fit(x_train,y_train)

MLPClassifier(alpha=0.01, batch_size=256, hidden_layer_sizes=(600, 300),
              learning_rate='adaptive', max_iter=500)

In [41]:
y_pred=model.predict(x_test)
accuracy=accuracy_score(y_true=y_test, y_pred=y_pred)
print("Accuracy: {:.2f}%".format(accuracy*100))

Accuracy: 63.54%


Seems good for now but will try to use PyTorch for building a ConvNet and see the results